import requests, json
from collections import OrderedDict
from operator import getitem
import time
import pandas as pd
import numpy as np
import math
import plotly
import plotly.graph_objects as go

In [91]:
headers = {
    "accept": "application/json",
    "x-amberdata-blockchain-id": "ethereum-mainnet",
    "x-api-key": "AMBER_DATA_API_KEY"
}

In [82]:
#Empyt dict for storing returned data. 
collection = {}
id = 0

#For Pagination
pagination = 0
paginationIncrement = 50
total_records_required = 2000
address = '0xCD531Ae9EFCCE479654c4926dec5F6209531Ca7b' #Wallet address of Interest
token_address = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48' #Token to analyze flow

## Collect latest token transfers to & from our target wallet

In [83]:
while pagination <= total_records_required:
    #Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{address}/token-transfers?page={pagination}&size=50'
    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)
    batch_list = json.loads(response.text)["payload"]['records']
    for item in batch_list:
        try: 
            id +=1
            collection[id] = {}
            collection[id]['from'] = item['from']['address']
            collection[id]['to'] = item['to'][0]['address']
            collection[id]['tokenAddress'] = item['tokenAddress'] 
            collection[id]['amount'] = float(item['amount']) / math.pow(10,6)
        except NameError:
            print(NameError)
        except:
            pass
    print('Finished page',pagination)
    pagination += paginationIncrement

Finished page 0
Finished page 50
Finished page 100
Finished page 150
Finished page 200
Finished page 250
Finished page 300
Finished page 350
Finished page 400
Finished page 450
Finished page 500
Finished page 550
Finished page 600
Finished page 650
Finished page 700
Finished page 750
Finished page 800
Finished page 850
Finished page 900
Finished page 950
Finished page 1000
Finished page 1050
Finished page 1100
Finished page 1150
Finished page 1200
Finished page 1250
Finished page 1300
Finished page 1350
Finished page 1400
Finished page 1450
Finished page 1500
Finished page 1550
Finished page 1600
Finished page 1650
Finished page 1700
Finished page 1750
Finished page 1800
Finished page 1850
Finished page 1900
Finished page 1950
Finished page 2000


## Cleaning data and filtering for only USDC transfers

In [85]:
df = pd.DataFrame.from_dict(collection,orient='index')
df['amount'] = df['amount'].astype(float)
#Filtering out just USD Transfers
df = df[df['tokenAddress'] =='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48']

## Creating a function to Plot our Sanky Chart

In [88]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

## Ploting our Sankey Chart

In [92]:
fig = genSankey(df, cat_cols = ['from', 'to'], value_cols='amount', title="USDC Flow to and from 0xCD531Ae9EFCCE479654c4926dec5F6209531Ca7b")
plotly.offline.plot(fig, validate=False)

'temp-plot.html'